In [ ]:
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers

In [1]:
import warnings
warnings.filterwarnings('ignore')

# 設定 APIKEY
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# **Intorduction to 🤗 Hugging Face and Transformers Library**

[🤗 hugging face NLP course](https://huggingface.co/learn/nlp-course/chapter1/1)


NLP 是語言學和機器學習領域，專注於理解與人類語言相關的一切。 NLP 任務的目標不僅是單獨理解單字，而且能夠理解這些單字的上下文。</p>
NLP 常見的任務有：
   1. **Classifying whole sentences 文本分類（分類整句）**：將整個句子進行分類
      - 情感分析：「這部電影很棒！」 -> positive
      - 垃圾郵件檢測：「你的 iphone 已被嚴重損壞」-> spam 
   2. **Classifying each word in a sentence 單詞分類**：對一句話中的所有字進行分類
      - 語法分析：「他跑得快」-> 他（代詞）跑（動詞）得（副詞）快（形容詞）
      - 命名實體 NER
   3. **Sentence Generation**
      1. **填充遮蔽詞**：「像這種要求，我這輩子[mask]！」-> [mask] 預測為 沒聽過
      2. **自動生成**：「今天天氣如何」-> 「今天天氣非常晴朗適合外出」
      3. **翻譯**：「你好」-> 「Hello」
      4. **摘要（問答）**
         1. 從文本中提取答案 Extractive QA：「法國的首都是巴黎。   首都在哪」-> 巴黎（藉由巴黎在原文的 index 抓出來）
         2. 以生成模型進行摘要 Generative QA：「法國的首都是巴黎。   首都在哪」-> 首都在巴黎（使用生成模型，例如 chatgpt）

## **`pipeline` in Hugging Face transformers**
``transformers`` 為 🤗Hugging face 提供的套件，讓開發者可以創建、使用 Hugging face hub 上 NLP、LLM 的模型</p>
> Hugging face hub 上的模型不只有 transformer，任何人都可以上傳任何類型的模型或資料集

在 `transformers` 中最高階的函數是 `pipeline`，
該函數將使用模型需要的預處理、推理與後處理串連起來，</p>
傳入指定的 task，`pipeline` 會自動以適合的模型進行推理（預測）。

可以從 [hub](https://huggingface.co/models) 透過 Tasks、Languages 篩選找到自己想要應用的模型</p>
從 1. [task summary](https://huggingface.co/docs/transformers/task_summary) 2. [Tasks](https://huggingface.co/tasks) 找到支援的 NLP 相關任務

### **使用 pipeline 完成常見 NLP 任務**

1. 情感分析 aka 分類問題

In [3]:
from pprint import pprint
from transformers import pipeline
pipe = pipeline(task="sentiment-analysis")
pipe("this is awesome!!!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998723268508911}]

2. 命名實體</p>



在 ner 任務中，模型會對所有字詞（token） 進行分類，得到：</p>
1. 該字詞（token）對應的 entity
2. score 機率值
3. 以及對應到文本的起始結束位置

In [ ]:
ner_pipe = pipeline(task="ner", 
                # model='dslim/bert-base-NER'
                )
ner_pipe("Hugging Face is a French company based in New York City.")

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [2]:
# TODO
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad, 
# 其為 Extractive QA 類摘要模型

from transformers import pipeline

### **利用 Conversation class 與 text-generation model 實作 chatbot**

In [ ]:
from pprint import pprint

from torch import cuda, bfloat16
from transformers import pipeline
from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的可以參考 Hugging face 官方文件～</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`。

In [ ]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

In [3]:
chatbot = pipeline(
    "text-generation", 
    model=hf_model, 
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

聊天式模型，例如 ChatGPT 其實基本上是透過 text-generation 作為基礎模型，進一步訓練模型能過聊天。
所以模型的選擇，我們可以在 Huggingface 上找到 text-generation 任務的模型，應該都可以支援。</p>

比較特別的是，要做聊天任務時，模型需要一些特殊符號來區別每一段訊息是來自於 User 或是 AI 還是 System Prompt</p>
而各個模型的特殊符號不盡相同，需要去查閱官方文件。例如 Demo 使用的聯發科 Breeze 模型是透過 `[INST]` 、 `[/INST]` 以及 `<s>` 作為區隔。
所以我們在使用模型時，就會需要將文字加上這些特殊符號才能夠發揮模型聊天的能力。</p>

通常我們會使用 list of dict 的方式處存聊天的記錄，使用 role 區別 user 與 ai，content 代表內容，而 Hugging face 的模型也支援這樣的格式，例如：


```
[
    {"role": "user", "content": "嗨你好嗎"},
    {"role": "assistant", "content": "嗨您好，我是您的 AI 助理，很高興為您服務。"},
    {"role": "user", "content": "掰掰"},
    {"role": "assistant", "content": "掰掰，期待再相見"},

]

```

我們可以透過實例化 Conversation 這個 class，透過 `add_user_input` 與 `append_response` 新增歷史使用者輸入與模型回覆，將資料變為上述的資料格式再送給模型進行推理。


In [4]:
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

利用 `add_user_input` 新增 user 聊天記錄

In [5]:
conversation.add_user_input("provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?")
print(f"目前聊天記錄：{conversation.messages}") # conversation.messages 可以直接丟給 chatbot 得到回覆

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}]


In [6]:
# 將 conversation.messages 丟給 chatbot
chatbot_result = chatbot(conversation.messages)
print(chatbot_result)

[{'generated_text': '根據提供的信息，repo的名称是"bert-base-uncased"。'}]


將 chatbot 的回覆以 `append_respons` 的方法加入 conversation 中

In [7]:
conversation.append_response(chatbot_result[0]['generated_text'])
print(f"目前聊天記錄：{conversation.messages}")

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}]


In [8]:
conversation.add_user_input("那什麼是 bert?")

print(f"目前聊天記錄：{conversation.messages}")

目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}, {'role': 'user', 'content': '那什麼是 bert?'}]


In [9]:
chatbot_result = chatbot(conversation.messages)
print(f"LLM 回覆：{chatbot_result}")

print("-"*10)
conversation.append_response(chatbot_result[0]['generated_text'])
print(f"目前聊天記錄：{conversation.messages}")

LLM 回覆：[{'generated_text': ' Bert是Bidirectional Embedding Representations from Transformers的缩写，是Google的一个自然语言处理模型。它基于Transformers架构，可以对文本数据学习语义表达，用于各种自然语言处理任务，如情感分析、命名实体识别、情况语言模型等。Bert模型通过预训练在大量的非任务专用数据上获得知识，然后在特定的任务上进行微调。'}]
----------
目前聊天記錄：[{'role': 'user', 'content': 'provided information: the name of repo is bert-base-uncased. Based on the provided information, what is the name of repo?'}, {'role': 'assistant', 'content': '根據提供的信息，repo的名称是"bert-base-uncased"。'}, {'role': 'user', 'content': '那什麼是 bert?'}, {'role': 'assistant', 'content': ' Bert是Bidirectional Embedding Representations from Transformers的缩写，是Google的一个自然语言处理模型。它基于Transformers架构，可以对文本数据学习语义表达，用于各种自然语言处理任务，如情感分析、命名实体识别、情况语言模型等。Bert模型通过预训练在大量的非任务专用数据上获得知识，然后在特定的任务上进行微调。'}]


### **embedding model (feature extraction)**
[參考](https://huggingface.co/tasks/feature-extraction)


Embedding 是將文字轉換成向量的技術，使得文字可以在數學空間中表示。</p>這些向量捕捉了文字之間的語義關係，使得相似的文字在向量空間中更接近。常見的嵌入模型包括 Word2Vec、GloVe 和 BERT 等。

在 Retrieval-Augmented Generation (RAG) 中，我們會用 Embedding model 用來將查詢（query）和候選文檔（document）轉換成向量。</p>
通過計算這些向量的相似度，可以找出與查詢最相關的文檔。這些相關文檔隨後用來生成回答，增強生成模型的準確性和上下文相關性。

我們使用 sentence transformers 這個套件，可以從 [官方文檔](https://sbert.net/docs/pretrained_models.html) 尋找自己希望使用的 model，也可以在 hugging face 平台上搜尋支援 feature-extraction 的模型。</p>

另外 hugging face 也提供 [embedding model](https://huggingface.co/spaces/mteb/leaderboard) 的排行榜給大家參考

In [10]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

/opt/conda/envs/verdict/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
# 利用 encode 得到 sentence 的 embedding
embedding_model.encode("哈囉，這是一個句子")

array([ 0.03142083, -0.01894771, -0.00766944, ..., -0.02039895,
       -0.01210877,  0.03742645], dtype=float32)

In [16]:
query = "為什麼 ML 需要做正規化"

source_sentence = [
    'Regularization is important!',
    'Dropout is important!',
    'Missing Data Handling is important!'
]

當我們有每個句子的 embedding 後就可以透過 cosine similarity 計算每個文本的相似度。

In [17]:
import numpy as np
def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [28]:
most_related_sentence = None
max_similarity = 0

for sentence in source_sentence:
    sim = calculate_cosine_similarity(
    embedding_model.encode(query),
    embedding_model.encode(sentence)
    )
    
    if sim > max_similarity:
        most_related_sentence = sentence
        max_similarity = sim
        
    print(f"{query} vs {sentence} similarity: {sim}")
    
print("="*10)
print(f"與「{query}」最相似文本：{most_related_sentence}")

為什麼 ML 需要做正規化 vs Regularization is important! similarity: 0.8665976524353027
為什麼 ML 需要做正規化 vs Dropout is important! similarity: 0.8055095672607422
為什麼 ML 需要做正規化 vs Missing Data Handling is important! similarity: 0.8144524693489075
與「為什麼 ML 需要做正規化」最相似文本：Regularization is important!


**作業 demo**

利用 Hugging Face 的 text-generation model 與 Sentence Transformers embedding model 實作 QA 檢索聊天機器人。</p>
基於提供的資料集，使用 Embedding Cosine Similarity 檢索參考資料，再透過 LLM 生成答案。</p>

1. Baseline
   - 將 demo 中的資料，替換成我們提供 or 自己的資料集
   - 能夠檢索相似資料
   - 基於檢索的資料進行回答
2. Advanced（Optional）
   - Embedding 怎麼儲存？每次都要重新計算嗎？
   - 該如何處理太久以前的歷史資料？
   - 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [19]:
qa_data = [
    """1. 入營注意事項：役男入營前無須先行理髮，待入營後統一安排理髮，再向役男收取費用
    2. 役男可攜帶手機(不得為大陸廠牌，且不提供充電)，部隊會集中保管，於每日夜間以定時定點方式使用。
    3. 營區不提供充電，建議攜帶拋棄式(手動)刮鬍刀；不可攜帶噴霧式液體。
    4. 役男可配戴隱形眼鏡，但因基礎訓練期間生活緊湊，而隱形眼鏡需相當時間消毒清洗，建議配戴鏡片眼鏡為宜，並可多備一副眼鏡，以供替換。
    """,
    
    """報到入營時應該文件：
    1. 徵集令。
    2. 役男本人之國民身分證正本。
    3. 私章。
    4. 健保ＩＣ卡。
    5. 郵局存摺正面影本。
    6. 替代役役男輔導需求調查表。
    7. 個人特殊醫療用品。
    """,
    
    """
    替代役訓練天數
    基礎訓練：21天(含撥交日)。(自253梯次起修正)
    專業訓練：以各分發需用機關不同而有個不同期間的專業訓練。
    """,
    
    """
    折抵役期規定
    請備妥相關證明文件，如高中（職）以上各級學校經軍訓主管驗證加註折抵役期日數之成績單、大專集訓結訓證書、
    驗退（停役）證明書或軍事學校退學（開除）證明書等正本（驗證後退還）。82年次以前出生者，
    合計不得逾30日，83-93年次以後出生者，合計不得逾15日。已受軍事入伍訓練者，請於收到徵集令時，向戶籍地區公所提出免受基礎訓練申請。
    """,
    
    """
    要當兵時，健保要辦理轉出轉入嗎?
    即將入營服常備兵役之役男需持徵集令向全民健保加保單位辦理轉出，轉出日期填報「入營當月份」，轉入單位免填，後續將由國軍單位接續辦理後續轉入程序。
    """,
    
    """當兵期間義務役薪水的入帳戶一定要郵局嗎?
    依現行規定目前有郵局、台新、土地銀行跟合作金庫這四家金融機構可使用。"""
]

In [20]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str]):
    most_related_sentence = None
    max_similarity = 0

    for sentence in source:
        sim = calculate_cosine_similarity(
        embedding_model.encode(query),
        embedding_model.encode(sentence)
        )
        
        if sim > max_similarity:
            most_related_sentence = sentence
            max_similarity = sim
            
    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [24]:
user_query = input(">>>")
conversation = Conversation()

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)
    
    user_query = input(">>>")
    

user: 我明天要去軍營報到，我應該要帶什麼文件
AI: 明天你要去軍營報到，你應該要帶以下文件：

1. 徵集令
2. 役男本人之國民身分證正本
3. 私章
4. 健保IC卡
5. 郵局存摺正面影本
6. 替代役役男輔導需求調查表
7. 個人特殊醫療用品

以上文件請務必備妥並確認後，明天前往軍營進行報到。
